In [1]:
pip install opencv-python


Note: you may need to restart the kernel to use updated packages.


In [9]:
import numpy as np
import pandas as pd
import cv2 as cv
import cv2 as cv2
from matplotlib import pyplot as plt
import math
import os
import shutil
from pathlib import Path
import glob
import itertools
from itertools import cycle
from scipy import interp

from statistics import mean
from statistics import stdev




In [10]:
pip install pyradiomics


  Using cached pyradiomics-3.0.1.tar.gz (34.5 MB)
Note: you may need to restart the kernel to use updated packages.


  ERROR: Command errored out with exit status 1:
   command: 'C:\Users\91700\anaconda3\python.exe' -u -c 'import io, os, sys, setuptools, tokenize; sys.argv[0] = '"'"'C:\\Users\\91700\\AppData\\Local\\Temp\\pip-install-ymfj4r0p\\pyradiomics_f5fce7430ea944cf8f20b0824c10127f\\setup.py'"'"'; __file__='"'"'C:\\Users\\91700\\AppData\\Local\\Temp\\pip-install-ymfj4r0p\\pyradiomics_f5fce7430ea944cf8f20b0824c10127f\\setup.py'"'"';f = getattr(tokenize, '"'"'open'"'"', open)(__file__) if os.path.exists(__file__) else io.StringIO('"'"'from setuptools import setup; setup()'"'"');code = f.read().replace('"'"'\r\n'"'"', '"'"'\n'"'"');f.close();exec(compile(code, __file__, '"'"'exec'"'"'))' bdist_wheel -d 'C:\Users\91700\AppData\Local\Temp\pip-wheel-3ht4nf4e'
       cwd: C:\Users\91700\AppData\Local\Temp\pip-install-ymfj4r0p\pyradiomics_f5fce7430ea944cf8f20b0824c10127f\
  Complete output (36 lines):
  C:\Users\91700\anaconda3\lib\site-packages\setuptools\dist.py:717: UserWarning: Usage of dash-separa

  Running setup.py clean for pyradiomics
Failed to build pyradiomics
    Running setup.py install for pyradiomics: started
    Running setup.py install for pyradiomics: finished with status 'error'


  creating build\lib.win-amd64-3.9\radiomics\schemas
  copying radiomics\schemas\paramSchema.yaml -> build\lib.win-amd64-3.9\radiomics\schemas
  copying radiomics\schemas\schemaFuncs.py -> build\lib.win-amd64-3.9\radiomics\schemas
  UPDATING build\lib.win-amd64-3.9\radiomics/_version.py
  set build\lib.win-amd64-3.9\radiomics/_version.py to 'v3.0.1'
  running build_ext
  building 'radiomics._cmatrices' extension
  error: Microsoft Visual C++ 14.0 or greater is required. Get it with "Microsoft C++ Build Tools": https://visualstudio.microsoft.com/visual-cpp-build-tools/
  ----------------------------------------
  ERROR: Failed building wheel for pyradiomics
    ERROR: Command errored out with exit status 1:
     command: 'C:\Users\91700\anaconda3\python.exe' -u -c 'import io, os, sys, setuptools, tokenize; sys.argv[0] = '"'"'C:\\Users\\91700\\AppData\\Local\\Temp\\pip-install-ymfj4r0p\\pyradiomics_f5fce7430ea944cf8f20b0824c10127f\\setup.py'"'"'; __file__='"'"'C:\\Users\\91700\\AppData\\

In [14]:

import radiomics
import SimpleITK as sitk
import six
from skimage.feature import greycomatrix, greycoprops
from array import *
from scipy.cluster.vq import whiten
from scipy.cluster.vq import kmeans

from sklearn.impute import SimpleImputer
from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import label_binarize
from sklearn.decomposition import PCA

from sklearn import datasets
from sklearn import svm
from sklearn.ensemble import RandomForestClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.multiclass import OneVsRestClassifier

from sklearn.model_selection import train_test_split
from sklearn.model_selection import KFold
from sklearn.model_selection import GridSearchCV

from sklearn.metrics import accuracy_score
from sklearn.metrics import precision_recall_fscore_support
from sklearn.metrics import confusion_matrix
from sklearn.metrics import classification_report
from sklearn.metrics import roc_auc_score
from sklearn.metrics import roc_curve, auc
from sklearn.metrics import average_precision_score
from sklearn.metrics import precision_recall_curve
from sklearn.metrics import plot_precision_recall_curve

import matplotlib.pyplot as plt
import matplotlib.image as img

ModuleNotFoundError: No module named 'radiomics'

In [5]:
oriBS = 'plant_data/Bacterial_Spot/'
oriEB = 'plant_data/Early_Blight/'
oriHL = 'plant_data/Healthy/'
oriLB = 'plant_data/Late_Blight/'
oriSLS = 'plant_data/Septorial_leaf_spot/'
oriYCV = 'plant_data/Yellow_Curl_Virus/'

# Folder path for segmented images
segBS = '/Bacterial_Spot_Segmented/'
segEB = '/Early_Blight_Segmented/'
segHL = '/Healthy_Segmented/'
segLB = '/Late_Blight_Segmented/'
segSLS = '/Septorial_Leaf_Spot_Segmented/'
segYCV = '/Yellow_Curl_Virus_Segmented/'


# File path for extracted features
feaBS = '/Bacterial_Spot_Features.csv'
feaEB = '/Early_Blight_Features.csv'
feaHL = '/Healthy_Features.csv'
feaLB = '/Late_Blight_Features.csv'
feaSLS = '/Septorial_Leaf_Spot_Features.csv'
feaYCV = '/Yellow_Curl_Virus_Features.csv'

In [6]:
os.makedirs(segBS)
os.makedirs(segEB)
os.makedirs(segHL)
os.makedirs(segLB)
os.makedirs(segSLS)
os.makedirs(segYCV)

In [7]:
def segmentation(originalImageFolder, newFolderPath, cannyMin, cannyMax, numOfControusThreshold, lowerBoundDenominator, dilationIt, erodeIt, kernelDil, kernelEro):

    fileList = []
    fileIDList = []

    fileIDList = next(os.walk(originalImageFolder), (None, None, []))[2] 
        
    print('Number of Images: ', len(fileIDList))
    i = 0
    for file in fileIDList:
        imgOriginal = cv.imread(originalImageFolder+file,1)
        imgGray = cv.imread(originalImageFolder+file,0)
        
        
        # Apply Gaussian Blur
        imgGray = cv.GaussianBlur(imgGray,(3,3),0)

        
        # Apply Canny edge
        edges = cv.Canny(imgGray,cannyMin,cannyMax)


        # Apply filter contour by length
        contours, hierarchy = cv2.findContours(edges,cv2.RETR_TREE,cv2.CHAIN_APPROX_NONE)
        contours = sorted(contours, key=lambda x: cv2.arcLength(x, True))
        if len(contours) > numOfControusThreshold:
            imgContours = np.zeros(imgGray.shape, dtype=np.uint8)
            lowerBound = max(numOfControusThreshold, int(len(contours)/lowerBoundDenominator))
            for j in reversed(range(len(contours) - lowerBound,len(contours))):
                length = cv2.arcLength(contours[j], True)
                cv2.drawContours(imgContours, contours, j, (255), -1)
        else:
            imgContours = edges

            
        # Apply dilation
        kernel = cv2.getStructuringElement(cv2.MORPH_ELLIPSE, (kernelDil,kernelDil))
        dilation = cv2.dilate(imgContours,kernel,iterations = dilationIt)


        # Find the max contours
        contours, hierarchy = cv2.findContours(dilation,cv2.RETR_TREE,cv2.CHAIN_APPROX_NONE)
        maxContour = max(contours, key = len)


        # Draw it on image
        img = np.zeros( imgGray.shape, dtype=np.uint8)
        img = cv2.drawContours(img, [maxContour], -1, (255),-1)


        # Erosion after filling contour
        kernel = cv2.getStructuringElement(cv2.MORPH_ELLIPSE, (kernelEro,kernelEro))
        img = cv2.erode(img,kernel,iterations = erodeIt)
        segmented = cv2.bitwise_and(imgOriginal,imgOriginal,mask = img)
        lab = cv2.cvtColor(segmented, cv2.COLOR_BGR2LAB)
        lab_planes = cv2.split(lab)
        
        
        # Apply CLAHE
        clahe = cv2.createCLAHE(clipLimit=5.0)
        lab_planes[0] = clahe.apply(lab_planes[0])
        lab = cv2.merge(lab_planes)

        
        # Convert back from RGB to LAB colour space
        bgr = cv2.cvtColor(lab, cv2.COLOR_LAB2BGR)

        
        # Write image into the folder
        cv2.imwrite(newFolderPath + fileIDList[i], bgr)
        i+=1
        
    # Create a zip file fir the folder to be downloaded
    shutil.make_archive(newFolderPath, 'zip', newFolderPath)

In [8]:
cannyMin = 40
cannyMax = 125
numOfControusThreshold = 70
lowerBoundDenominator = 2
dilationIt = 20
erodeIt = 15
kernelDil = 3
kernelEro = 3
segmentation(oriBS, segBS, cannyMin, cannyMax, numOfControusThreshold, \
             lowerBoundDenominator, dilationIt, erodeIt, kernelDil, kernelEro)


cannyMin = 80
cannyMax = 125
numOfControusThreshold = 50
lowerBoundDenominator = 2
dilationIt = 20
erodeIt = 15
segmentation(oriEB, segEB, cannyMin, cannyMax, numOfControusThreshold, \
             lowerBoundDenominator, dilationIt, erodeIt, kernelDil, kernelEro)


cannyMin = 80
cannyMax = 125
numOfControusThreshold = 30
lowerBoundDenominator = 2
dilationIt = 20
erodeIt = 15
segmentation(oriHL, segHL, cannyMin, cannyMax, numOfControusThreshold, \
             lowerBoundDenominator, dilationIt, erodeIt, kernelDil, kernelEro)


cannyMin = 60
cannyMax = 125
numOfControusThreshold = 70
lowerBoundDenominator = 2
dilationIt = 20
erodeIt = 15

segmentation(oriLB, segLB, cannyMin, cannyMax, numOfControusThreshold, \
             lowerBoundDenominator, dilationIt, erodeIt, kernelDil, kernelEro)


cannyMin = 100
cannyMax = 125
numOfControusThreshold = 100
lowerBoundDenominator = 4
dilationIt = 30
erodeIt = 20

segmentation(oriSLS, segSLS, cannyMin, cannyMax, numOfControusThreshold, \
             lowerBoundDenominator, dilationIt, erodeIt, kernelDil, kernelEro)
cannyMin = 40
cannyMax = 125
numOfControusThreshold = 40
lowerBoundDenominator = 4
dilationIt = 30
erodeIt = 20

segmentation(oriYCV, segYCV, cannyMin, cannyMax, numOfControusThreshold, \
             lowerBoundDenominator, dilationIt, erodeIt, kernelDil, kernelEro)

Number of Images:  2


error: OpenCV(4.6.0) D:\a\opencv-python\opencv-python\opencv\modules\imgproc\src\smooth.dispatch.cpp:617: error: (-215:Assertion failed) !_src.empty() in function 'cv::GaussianBlur'
